In [20]:
import Pkg
Pkg.activate("../../../FinancialPlanner")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner`


In [21]:
using Revise
using Turing
using LinearAlgebra
using Distributions
using Random
using StatsPlots
using PDMats
using StatsBase
using Base.Iterators
using DataFrames, TimeSeries, XLSX
using PrettyTables

In [22]:
include("../VARs/utils.jl")
includet("../bootstrap_model.jl")

In [23]:
df = DataFrame(XLSX.readtable("../../data/usa_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.month, last)

freq = 12
returns = percentchange(data_source[:Total_Stock, :Price_10Y], :log)
cpi = percentchange(data_source[:CPI], :log)
real_returns = returns .- cpi
start = Date(1955, 01,01)
end_d = Date(2024, 12, 31)


display("Returns")
display(returns_summarystats(to(from(returns, start),end_d), freq))
display("Real_returns")
display(returns_summarystats(to(from(real_returns, start),end_d), freq))


display("Correlations")
display(cor(values(to(from(real_returns, start),end_d)))) 

"Returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock,0.1473,1.8543,0.1012,0.4365,-2.9008,-0.6469,-0.1883,0.0258,2.1941,0.6874,0.1474
Price_10Y,0.0755,1.3969,0.0531,0.1909,-0.9866,0.3097,-0.0893,0.1022,1.9867,0.7037,0.0377


nothing

"Real_returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_CPI,0.1479,1.7607,0.0658,0.4053,-2.9112,-0.596,-0.2186,0.032,1.9119,0.445,0.1107
Price_10Y_CPI,0.078,1.279,0.0177,0.1544,-1.1677,0.2268,-0.1322,0.1437,2.0153,0.2268,0.0058


nothing

"Correlations"

2×2 Matrix{Float64}:
 1.0        0.0980477
 0.0980477  1.0

In [24]:
assets_names = [:Total_Stock_CPI, :Price_10Y_CPI]
data = to(from(real_returns[assets_names], start),end_d) 

scenarios = block_bootstrap(values(data), 10_000, block_size=1)

periods = [1,5, 10, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))
n_assets = length(assets_names)
for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.01, .025, .05, .25, .5, .75, .95, .975, .99], string.(periods), string.(assets_names[a]))
end  


,Total_Stock_CPI,Price_10Y_CPI
1,0.0656,0.0184
5,0.0656,0.0176
10,0.0662,0.0176
25,0.0662,0.0176


,Total_Stock_CPI,Price_10Y_CPI
1,0.148,0.0782
5,0.0671,0.0349
10,0.0471,0.0246
25,0.0295,0.0156


,Total_Stock_CPI,Price_10Y_CPI
1,-0.16,0.1069
5,-0.049,0.0384
10,-0.0463,0.0116
25,-0.0016,0.0344


,Total_Stock_CPI,Price_10Y_CPI
1,0.1767,0.2572
5,0.0602,0.0784
10,0.0066,-0.0217
25,-0.0378,0.0124


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.3014,-0.2362,-0.184,-0.0296,0.0698,0.1656,0.3037,0.3459,0.3985
5,-0.0965,-0.0707,-0.0453,0.0211,0.0661,0.1114,0.1741,0.1957,0.2201
10,-0.0473,-0.0273,-0.0115,0.0344,0.0667,0.0979,0.1434,0.1573,0.1757
25,-0.0012,0.0081,0.0172,0.0464,0.0663,0.0858,0.1145,0.124,0.1356


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.1618,-0.1326,-0.1078,-0.0335,0.0176,0.0687,0.1498,0.1788,0.2101
5,-0.0643,-0.0516,-0.0398,-0.0054,0.0175,0.0406,0.0749,0.0867,0.1017
10,-0.0394,-0.0306,-0.0225,0.0008,0.0174,0.0343,0.058,0.0657,0.0749
25,-0.018,-0.013,-0.008,0.007,0.0176,0.0279,0.0435,0.0485,0.0544


In [25]:
mdd, mddl = max_drawdown_and_length(scenarios[1,:,:])

display(assets_names[1])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

mdd, mddl = max_drawdown_and_length(scenarios[2,:,:])

display(assets_names[2])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

:Total_Stock_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.30466  0.387786  0.448968  0.518741  0.666133

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 55.0  89.0  121.0  169.0  324.0

:Price_10Y_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.211377  0.283244  0.340507  0.411265  0.565475

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 90.0  158.0  225.0  329.0  640.0